In [1]:
import pickle
import gzip

#how be able to iterate through all of the different files

#updated
vals_dict = {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],
'n': [0.1, 0.25, 0.5, 0.63, 0.69, 0.75, 0.81, 0.9], 'k':[1,10,40,200]  }



In [ ]:
len(text)

In [ ]:
text[0].shape

In [ ]:
rand_selects

In [2]:
import numpy as np 
import pandas as pd
prompts=pd.read_csv('test_dataframe_500primer.csv') # the text has the prompts so dont need this. 

# Convert all of the numbers text from all of the different models into one dataframe with all of the prompts and the produced text separated out

In [3]:
text[0][0]

NameError: name 'text' is not defined

In [4]:
# used to decode:

from decodeLogits import *
batch = 0
ind = 0
'''
tokens = []
for j in range(text[0].shape[2]):
    tokens.append(np.argmax(df[0][ind, :, j]))'''
decoder_text(text[batch][ind])

NameError: name 'text' is not defined

## Generating all completions for a particular random prompt

In [5]:
 vals_dict.items()
    
I also need to store the sorted indices of the differnet words

SyntaxError: invalid syntax (<ipython-input-5-ff9ac135490b>, line 3)

In [47]:
decoder_text(text[0][0][0:100])

"[WP] You look out at the bright lights of the city and wonder how you ever grew up. \n The lights dance across the evening sky. You can see the buildings in the distance, some lights turning of as the stores close down. You wonder how this all happened, when did everything change? \n \n When did *you* change? \n \n Everyone is born a child. Some grow up faster than others. Some do n't grow up at all. Hours"

In [59]:
seed = 27
prompt_length = 100
gs=dict()
methods_wanted = ['tfs_0.95', 'n_0.69', 'k_40']
first = True
for key, params in vals_dict.items():
    for par in params:
        if key+'_'+str(par) in methods_wanted:
            #all_logits = pickle.load( gzip.open('gpt-2_output/all_logits_'+key+'-sampling-type_'+par+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
            text = pickle.load( gzip.open('gpt-2_output/all_text_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed.pickle.gz', 'rb'))
            rand_selects = pickle.load( gzip.open('gpt-2_output/prompt_rand_selections_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed.pickle.gz', 'rb'))

            generated_output = []
            for batch in text:
                for entry in batch:
                    generated_output.append(decoder_text(entry[prompt_length:]))
            col_name = 'Samp Type:'+key+' Param:'+str(par)
            gs[col_name] = generated_output
            
    if first==True:
        prompts_used = []
        for batch in text:
            for entry in batch:
                prompts_used.append(decoder_text(entry[0:prompt_length]))
        
        first=False
gs = pd.DataFrame(gs)

In [60]:
gs.shape

(100, 3)

In [61]:
gs.head()

,Samp Type:tfs Param:0.95,Samp Type:n Param:0.69,Samp Type:k Param:40
0,go by. Months go by. You can't remember the l...,"have passed, and yet you still feel the touch...","later, it's your turn.\n It's my turn. It se..."
1,I throw in my towel and sit around crying? It...,I rush out of there?\n\nThen I was suddenly p...,I just ignore that and jump right in? \n \n\n...
2,"the boy to lose his virginity, for it would b...",a child to marry. It was the magic of the cav...,the boy's wife and child and what could a man...
3,have taken her to the government. She doesn't...,you're here? It's not a coincidence. I'm sure...,"raised her. We're going to leave her here, an..."
4,"’ t didn't know who I was, that I ’ ve spoken...",", was one of the only ones of us who could und...",", ’ t resembled Bjørn Bjørnson in his youth, a..."


In [70]:
import numpy as np
select_a_random_prompt = np.random.choice(gs.shape[0])
select_random_generation_methods = np.random.choice(range(gs.shape[1]), size =gs.shape[1], replace=False )
#select_random_generation_methods = np.insert(select_random_generation_methods, 0, 0)
print(select_a_random_prompt)
select_random_generation_methods

16


array([0, 2, 1])

In [67]:
with open('test_blind_output3.txt', 'w') as file: 
    file.write('Prompt: \n \n')
    file.write(prompts_used[select_a_random_prompt])
    file.write('\n')
    file.write('================== \n')
    for ind, out in enumerate(gs.iloc[select_a_random_prompt,select_random_generation_methods].tolist()):
        file.write('Random Generation: '+str(ind)+' \n \n')
        file.write(out)
        
        file.write('\n')
        file.write('================== \n')
        if ind == gs.shape[1]-1: # dont write random generation at the end. 
            continue
        



Some of the very tight ones are too close and degenerate into repeats. The prompts and stochasticity of a single generation are really hard to deal with though. it is also a lot to read in order to evaluate the quality. 
What is something I can evaluate algorithmically and where diverse beam search is outperformed? It may inherently need to be something that is long. 
Seems like there is definitely a sweet spot where dont want too many options but also dont want it to be degenerate. 

In [68]:
print(select_a_random_prompt)
select_random_generation_methods

25


array([0, 1, 2])

In [69]:
gs.iloc[select_a_random_prompt,select_random_generation_methods]

Samp Type:tfs Param:0.95     life was about to change if she didn't give i...
Samp Type:n Param:0.69       heart stopped beating and the pounding of his...
Samp Type:k Param:40         mind was always busy, but he wouldn't be able...
Name: 25, dtype: object

In [ ]:
gs.iloc[select_a_random_prompt,0]

In [ ]:
choose_prompt =rand_selects[select_a_random_prompt]
choose_prompt

In [ ]:
prompts.head()

In [ ]:
print(prompts.iloc[choose_prompt].test_target)